In [ ]:
import batoid
import yaml
import os
import numpy as np
from ipywidgets import interact
import ipywidgets as widgets
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
filename = os.path.join(batoid.datadir, 'HSC', 'HSC3.yaml')
config = yaml.load(open(filename))
telescope = batoid.parse.parse_optic(config['opticalSystem'])

In [ ]:
# Sanity check optic shifting
@interact(optic=widgets.Dropdown(
              options=['SubaruHSC.HSC.G1',
                       'SubaruHSC.HSC.G2',                       
                       'SubaruHSC.HSC.ADC',
                       'SubaruHSC.HSC.ADC.ADC1',
                       'SubaruHSC.HSC.ADC.ADC2',
                       'SubaruHSC.HSC.G3',
                       'SubaruHSC.HSC.G4',
                       'SubaruHSC.HSC.G5',
                       'SubaruHSC.HSC.F',
                       'SubaruHSC.HSC.W',
                       'SubaruHSC.HSC.D'],
              default='SubaruHSC.HSC.ADC'
          ),
          dx=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.2),
          dy=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.0),
          dz=widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.0),
          dthx=widgets.FloatSlider(min=-90, max=90, step=5, value=0.0),
          dthy=widgets.FloatSlider(min=-90, max=90, step=5, value=0.0),
          dthz=widgets.FloatSlider(min=-90, max=90, step=5, value=0.0)
         )
def shiftOptic(optic, dx, dy, dz, dthx, dthy, dthz):
#     rot = batoid.RotX(np.pi/180*dthx) * batoid.RotY(np.pi/180*dthy) * batoid.RotZ(np.pi/180*dthz)
    rot = batoid.RotZ(np.pi/180*dthz) * batoid.RotX(np.pi/180*dthx) * batoid.RotY(np.pi/180*dthy)
    perturbedTelescope = telescope.withGloballyShiftedOptic(optic, batoid.Vec3(dx, dy, dz))
    perturbedTelescope = perturbedTelescope.withLocallyRotatedOptic(optic, rot)
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111, projection='3d')
    perturbedTelescope.itemDict['SubaruHSC.HSC'].draw3d(ax, c='k')
    #And redraw the perturbed optic in red
    perturbedTelescope.itemDict[optic].draw3d(ax, c='r')
    ax.set_xlim(-0.5, 0.5)
    ax.set_ylim(-0.5, 0.5)    
    ax.set_zlim(13.5, 15.25)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")    
    plt.show()